In [24]:
import pandas as pd
import os
import copy
import unittest
import string
import re

def read_file(trainFile):
    pwd = os.getcwd()
    os.chdir(os.path.dirname(trainFile))
    file1 = open(trainFile, 'r') 
    lines = file1.read().splitlines()
    return lines[0:3] + ["  #D#C#B#A#"] + ["  #D#B#A#C#"] + lines[3:6]

In [25]:
lines = read_file("C:/Code/advent/2021_day23_input.txt")

In [26]:
lines_example1 = read_file("C:/Code/advent/2021_day23_example1.txt")
lines_example1

['#############',
 '#...........#',
 '###B#C#B#D###',
 '  #D#C#B#A#',
 '  #D#B#A#C#',
 '  #A#D#C#A#',
 '  #########']

In [27]:
def parse(input):
    result = {}
    row = 0
    for line in input:
        col = 0
        for c in line:
            point = (col,row)
            if c != '#' and c != ' ':
                result[point] = c
            col += 1
        row += 1
    return result

def print_field(input):
    x_n = 0
    y_n = 0
    for point in input:
        x_n = max(x_n, point[0])
        y_n = max(y_n, point[1])
    for y in range(y_n+1):
        line = ''
        for x in range(x_n+1):
            line += input.get((x,y),' ')
        print(line)
    print('')

print_field(parse(lines_example1))

            
 ...........
   B C B D  
   D C B A  
   D B A C  
   A D C A  



In [28]:
print_field(parse(lines))
print(parse(lines))

            
 ...........
   B B D D  
   D C B A  
   D B A C  
   C C A A  

{(1, 1): '.', (2, 1): '.', (3, 1): '.', (4, 1): '.', (5, 1): '.', (6, 1): '.', (7, 1): '.', (8, 1): '.', (9, 1): '.', (10, 1): '.', (11, 1): '.', (3, 2): 'B', (5, 2): 'B', (7, 2): 'D', (9, 2): 'D', (3, 3): 'D', (5, 3): 'C', (7, 3): 'B', (9, 3): 'A', (3, 4): 'D', (5, 4): 'B', (7, 4): 'A', (9, 4): 'C', (3, 5): 'C', (5, 5): 'C', (7, 5): 'A', (9, 5): 'A'}


In [29]:
def create_map_template(input):
    result = {}
    for key in input:
        result[key] = '.'
    return result

map_template = create_map_template(parse(lines))
print_field(map_template)

            
 ...........
   . . . .  
   . . . .  
   . . . .  
   . . . .  



In [30]:
def energy_for_type(input):
    if input == 'a' or input == 'A':
        return 1
    if input == 'b' or input == 'B':
        return 10
    if input == 'c' or input == 'C':
        return 100
    if input == 'd' or input == 'D':
        return 1000

In [31]:
def create_route_next(start, end):
    if start[0] == end[0]:
        # move vertical
        if start[1] < end[1]:
            return (start[0],start[1]+1)
        elif start[1] > end[1]:            
            return (start[0],start[1]-1)
    elif start[1] == 1:
        # move horizontal
        if start[0] < end[0]:
            return (start[0]+1,start[1])
        elif start[0] > end[0]:
            return (start[0]-1,start[1])            
    else:
        # move in U
        return (start[0],start[1]-1)
    
    print(start,end)
    return 1 + 'should not reach'

def create_route(start, end):
    result = []
    next = start
    while next != end:
        next = create_route_next(next,end)
        result.append(next)
    return result

def get_valid_places_template(input):
    valid_places = copy.deepcopy(map_template)
    for i in range(0,4):
        valid_places.pop((3+2*i,1))
    return valid_places
print_field(get_valid_places_template(map_template))
valid_places = get_valid_places_template(map_template)

def create_all_routes():
    result = {}
    for start in valid_places:
        for end in valid_places:
            result[(start, end)] = create_route(start,end)
    return result

all_routes = create_all_routes()
print(len(all_routes))
print(all_routes[((3,5),(5,3))])

            
 .. . . . ..
   . . . .  
   . . . .  
   . . . .  
   . . . .  

529
[(3, 4), (3, 3), (3, 2), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3)]


In [35]:
def is_valid_move(map, start, end):
    route = all_routes[(start,end)]
    for point in route:
        if map[point] != '.':
            return False
    return True

def get_pieces(map):
    result = {}
    for point in map:
        value = map[point] 
        if value != '.':
            result[point] = value
    return result

def get_destination_x(animal):
    if animal == 'A': 
        return 3
    if animal == 'B':
        return 5
    if animal == 'C':
        return 7
    if animal == 'D':
        return 9

destination_x_map = {'A':3, 'B':5, 'C':7, 'D':9}

def is_solved(map, point):
    (x,y) = point
    if y == 1:
        return False
    piece = map[point]
    if x != destination_x_map[piece]:
        return False    
    if y>1:
        for below in range(y+1,6):  # hardcoded size of part two here
            if map[(x,below)] != piece:
                return False
    return True

def get_unsolved_pieces(map):
    pieces = get_pieces(map)
    result = []
    for point in pieces:
        if not is_solved(map, point):
            result.append(point)
    return result

def split_solved_unsolved(map):
    pieces = get_pieces(map)
    solved = []
    unsolved = []
    for point in pieces:
        if not is_solved(map, point):
            unsolved.append(point)
        else:
            solved.append(point)
    return solved, unsolved

print(get_unsolved_pieces(parse(lines)))
print(get_unsolved_pieces(parse(lines_example1)))

[(3, 2), (5, 2), (7, 2), (9, 2), (3, 3), (5, 3), (7, 3), (9, 3), (3, 4), (5, 4), (7, 4), (9, 4), (3, 5), (5, 5), (7, 5), (9, 5)]
[(3, 2), (5, 2), (7, 2), (9, 2), (3, 3), (5, 3), (7, 3), (9, 3), (3, 4), (5, 4), (7, 4), (9, 4), (5, 5), (9, 5)]


In [39]:
def calculate_lower_bound(input):
    solved,unsolved = split_solved_unsolved(input)
    counts = {}
    counts['A'] = 0
    counts['B'] = 0
    counts['C'] = 0
    counts['D'] = 0
    for point in solved:
        key = input[point]
        counts[key] += 1
    lower_bound = 0
    for point in unsolved:
        key = input[point]
        target = (destination_x_map[key], 5 - counts[key])
        cost = len(all_routes[(point,target)]) * energy_for_type(key)
        lower_bound += cost
        counts[key] += 1    
    return lower_bound

print(calculate_lower_bound(parse(lines)))
print(calculate_lower_bound(parse(lines_example1)))

32513
36041


In [49]:
def sort_key(state):
    return -state[1]

def sort_key_lowerbound(state):
    return -state[3]

def select_state_with_lowest_cost(states):
    highest = 1000000
    index = None
    for i in range(len(states)):
        m, score, depth, lb = states[i]
        if score < highest:
            index = i
            highest = score
    return index

def select_state_with_highest_depth(states):
    best_depth = -1
    best_cost = 1000000
    index = None
    for i in range(len(states)):
        m, score, depth, lb = states[i]
        if depth > best_depth:
            index = i
            best_depth = depth
            best_cost = score
        elif depth == best_depth:
            if score < best_cost:
                index = i
                best_depth = depth
                best_cost = score
    return index

def is_valid_destination(current_map, start,end, animal):
    if end[1] > 1:
        if not destination_x_map[animal] == end[0]:
            return False
        for below in range(end[1]+1,6): # hardcoded size of part 2
            if current_map[(end[0],below)] != animal:
                return False
        return True
    else:
        # going to parking position
        return start[1] > 1

def get_allowed_moves(current_map, piece):
    result = []
    for target in valid_places:
        if not is_valid_destination(current_map, piece, target, current_map[piece]):
            continue        
        if not is_valid_move(current_map, piece, target):
            continue        
        result.append(target)        
    return result

def move(current, piece, target):
    current_map, current_score, current_depth, current_lower_bound = current
    next_map = copy.deepcopy(current_map)    
    next_map[piece] = '.'
    next_map[target] = current_map[piece]
    cost = len(all_routes[(piece,target)]) * energy_for_type(current_map[piece])
    lower_bound = calculate_lower_bound(next_map) + cost + current_score
    return (next_map, cost+current_score, current_depth+1, lower_bound)

def calculate_part_two(input):
    starting_state = (input, 0, 0, calculate_lower_bound(input))
    remaining_states = [starting_state]
    max_iterations = 40000000
    n = 0
    best = None
    while len(remaining_states) > 0 and n < max_iterations:
        n+=1        
        #select state to investigate
        index = len(remaining_states) - 1
                
        #if len(remaining_states) < 50:
            #index = select_state_with_lowest_cost(remaining_states)
        #else:
            #index = select_state_with_highest_depth(remaining_states)        

        current = remaining_states.pop(index)         
        current_map, current_score, current_depth, current_lower_bound = current

        if n % 5000 == 0:
            min_index = select_state_with_lowest_cost(remaining_states)
            min_score = remaining_states[min_index][1]
            print('iteration', n, 'best', best[1] if best != None else None, 'min_score', min_score, 'score', current_score, 'current_lower_bound', current_lower_bound,'depth', current_depth,len(remaining_states))

        if best != None:
            if best[1] <= current_lower_bound:
                continue
        if current_lower_bound > 47481: # cheat a little?
            continue

        if (not best == None) and current_score > best[1]:
            # everything else is more expensive
            break
        current_unsolved = get_unsolved_pieces(current_map)
        if len(current_unsolved) == 0:
            # done with this one
            if best == None:
                best = current
            elif current_score < best[1]:
                best = current
        else:
            for piece in current_unsolved:
                targets = get_allowed_moves(current_map, piece)
                new_states = []
                for t in targets:
                    next = move(current, piece, t)                    
                    new_states.append(next)                    
                    #print_field(next[0])
                    #print('score',next[1], 'depth',next[2])
                    #print('-----------------------------')
                new_states.sort(key=sort_key_lowerbound)
                for state in new_states:
                    remaining_states.append(state)

    print_field(best[0])

    return best[1]
    
calculate_part_two(parse(lines_example1))

iteration 5000 best None min_score 20 score 2840 current_lower_bound 40861 depth 5 50
iteration 10000 best None min_score 20 score 8814 current_lower_bound 40443 depth 15 58
iteration 15000 best None min_score 20 score 13674 current_lower_bound 47101 depth 15 58
iteration 20000 best None min_score 20 score 15270 current_lower_bound 47101 depth 12 63


KeyboardInterrupt: 

In [50]:
# 47481 too high @ 600000 3 minutes
# 46721

calculate_part_two(parse(lines))

iteration 5000 best None min_score 20 score 5010 current_lower_bound 39517 depth 4 61
iteration 10000 best None min_score 20 score 7593 current_lower_bound 44033 depth 12 62
iteration 15000 best None min_score 20 score 6270 current_lower_bound 36913 depth 9 55
iteration 20000 best None min_score 20 score 7073 current_lower_bound 40615 depth 6 46
iteration 25000 best None min_score 20 score 2814 current_lower_bound 37077 depth 7 53
iteration 30000 best None min_score 20 score 6443 current_lower_bound 38733 depth 5 67
iteration 35000 best None min_score 20 score 8442 current_lower_bound 42753 depth 7 55
iteration 40000 best None min_score 20 score 4503 current_lower_bound 40055 depth 7 66
iteration 45000 best None min_score 20 score 13943 current_lower_bound 40035 depth 12 79
iteration 50000 best None min_score 20 score 10463 current_lower_bound 46555 depth 7 67
iteration 55000 best None min_score 20 score 13437 current_lower_bound 38737 depth 8 78
iteration 60000 best None min_score 20 

KeyboardInterrupt: 